In [40]:
!pip install pymupdf

  Obtaining dependency information for pymupdf from https://files.pythonhosted.org/packages/e3/65/1bb8e3e39f2eb233819ba9119fffc2fb8a947e0a81ed7a3d5336f7182ca1/PyMuPDF-1.24.8-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for PyMuPDFb==1.24.8 from https://files.pythonhosted.org/packages/e9/cd/446d9bd190b79d18c92363f572bd45a13abd442ef63cb3578414b3bbc510/PyMuPDFb-1.24.8-py3-none-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.3 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 8.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [41]:
import snowflake.connector
import boto3
from botocore.exceptions import ClientError
from datetime import datetime
import fitz  # PyMuPDF

# Snowflake connection parameters
snowflake_account = 'oytvmyv-as20412'
snowflake_user = 'AKSHAY'
snowflake_password = 'Srini@1406'
snowflake_database = 'unstructured'
snowflake_schema = 'unstructured'
snowflake_table = 'pdf_text_data'

# AWS S3 parameters
aws_access_key_id = ''
aws_secret_access_key = ''
s3_bucket_name = 'snowflakebucket16789'
file_key = 'Get_Started_With_Smallpdf.pdf'

# Initialize Snowflake connection
conn = snowflake.connector.connect(
    user=snowflake_user,
    password=snowflake_password,
    account=snowflake_account,
    database=snowflake_database,
    schema=snowflake_schema
)

# Initialize S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

try:
    # Download file from S3 as binary
    response = s3.get_object(Bucket=s3_bucket_name, Key=file_key)
    file_content = response['Body'].read()  # Read as binary data

    # Save the file locally
    with open('/tmp/temp_pdf.pdf', 'wb') as f:
        f.write(file_content)

    # Extract text from PDF
    pdf_document = fitz.open('/tmp/temp_pdf.pdf')
    text_content = ""
    for page in pdf_document:
        text_content += page.get_text()
    pdf_document.close()

    # Prepare Snowflake INSERT statement
    insert_stmt = f"INSERT INTO {snowflake_table} (filename, text, upload_date) VALUES (%s, %s, %s)"

    # Execute INSERT statement
    with conn.cursor() as cursor:
        cursor.execute(insert_stmt, (file_key, text_content, datetime.utcnow()))

    conn.commit()
    print("Data inserted successfully into Snowflake.")

except ClientError as e:
    print(f"Error: {e.response['Error']['Message']}")

finally:
    conn.close()



Data inserted successfully into Snowflake.
